# Extract Fast5 data

In [1]:
# packages
import numpy as np
import pandas as pd
import pickle

In [2]:
# load current data
with open('fast5_current_m6A_sams-345_100nt.pickle', 'rb') as f:
    current = pickle.load(f)
print(current.shape)

(373772, 305)


In [4]:
current.to_csv('fast5_current_m6A_sams-345_100nt.csv')

In [ ]:
# check fast5 structure

In [1]:
# load packages
import h5py
import numpy as np
import pandas as pd
import random
from tombo import tombo_helper
from tqdm import tqdm_notebook as tqdm

In [139]:
# load fast5
f5 = '/Volumes/DATA_3/190926_DrKuroyanagi_sams_345/190926_DrKuroyanagi_sams_345/20190926_0543_MN17290_FAK89523_0ee12f5a/fast5_pass_GUCAUGGG_single/1/ffff4209-d57a-433a-b194-1c5ff2a47c2a.fast5'
f5 = h5py.File(f5, 'r')

In [176]:
# check items
def printAll(name):
    print(name)

f5.visit(printAll)

Analyses
Analyses/Segmentation_000
Analyses/Segmentation_000/Summary
Analyses/Segmentation_000/Summary/segmentation
Analyses/Basecall_1D_000
Analyses/Basecall_1D_000/BaseCalled_template
Analyses/Basecall_1D_000/BaseCalled_template/Fastq
Analyses/RawGenomeCorrected_000
Analyses/RawGenomeCorrected_000/BaseCalled_template
Analyses/RawGenomeCorrected_000/BaseCalled_template/Alignment
Analyses/RawGenomeCorrected_000/BaseCalled_template/Events
Analyses/RawGenomeCorrected_sams345
Analyses/RawGenomeCorrected_sams345/BaseCalled_template
Analyses/RawGenomeCorrected_sams345/BaseCalled_template/Alignment
Analyses/RawGenomeCorrected_sams345/BaseCalled_template/Events
Raw
Raw/Reads
Raw/Reads/Read_62214
Raw/Reads/Read_62214/Signal
UniqueGlobalKey
UniqueGlobalKey/channel_id
UniqueGlobalKey/context_tags
UniqueGlobalKey/tracking_id


In [ ]:
# extract current

In [2]:
# fast5 file paths
fast5s = {'unm': '/Volumes/DATA_3/190926_DrKuroyanagi_sams_345/190926_DrKuroyanagi_sams_345/20190926_0543_MN17290_FAK89523_0ee12f5a/fast5_pass_single_unm',
          'm6A': '/Volumes/DATA_3/190926_DrKuroyanagi_sams_345/190926_DrKuroyanagi_sams_345/20190926_0543_MN17290_FAK89523_0ee12f5a/fast5_pass_single_m6A',
          'vivo': '/Volumes/DATA_2/sams/MinION/20190326_0351_190326_DrKuroyanagi_w12457/RAW/fast5',
          'unm long': '/Volumes/DATA_2/sams/MinION/20190326_0350_190326_DrKuroyanagi_sams3_sams4/RAW/fast5_filtered'
          }

# output range
region = 51

In [3]:
# for sams-3b and sams-5b
# plot sites
plotSites = {'sams-3b': 1262, 'sams-5b': 1315}

# tombo groups
tombo_groups = {'unm': 'RawGenomeCorrected_sams3b5',
                'm6A': 'RawGenomeCorrected_sams3b5',
                'vivo': 'RawGenomeCorrected_sams3abc4bdef5',
                'unm long': 'RawGenomeCorrected_sams3abc4bdef5'
                }

# csv file name
out_name = 'fast5_current_m6A_sams-3b5_100nt.csv'

In [9]:
# for sams-3c
# plot sites
plotSites = {'sams-3c': 1538, 'sams-5b': 1315}

# tombo groups
tombo_groups = {'unm': 'RawGenomeCorrected_sams3c5',
                'm6A': 'RawGenomeCorrected_sams3c5',
                'vivo': 'RawGenomeCorrected_sams3abc4bdef5',
                'unm long': 'RawGenomeCorrected_sams3abc4bdef5'
                }

# csv file name
out_name = 'fast5_current_m6A_sams-3c5_100nt.csv'

In [12]:
# for sams-4d
# plot sites
plotSites = {'sams-4d': 1265, 'sams-5b': 1315}

# tombo groups
tombo_groups = {'unm': 'RawGenomeCorrected_sams4d5',
                'm6A': 'RawGenomeCorrected_sams4d5',
                'vivo': 'RawGenomeCorrected_sams3abc4bdef5',
                'unm long': 'RawGenomeCorrected_sams3abc4bdef5'
                }

# csv file name
out_name = 'fast5_current_m6A_sams-4d5_100nt.csv'

In [14]:
# for sams-4e
# plot sites
plotSites = {'sams-4e': 1388, 'sams-5b': 1315}

# tombo groups
tombo_groups = {'unm': 'RawGenomeCorrected_sams4e5',
                'm6A': 'RawGenomeCorrected_sams4e5',
                'vivo': 'RawGenomeCorrected_sams3abc4bdef5',
                'unm long': 'RawGenomeCorrected_sams3abc4bdef5'
                }

# csv file name
out_name = 'fast5_current_m6A_sams-4e5_100nt.csv'

In [16]:
# for sams-4f
# plot sites
plotSites = {'sams-4f': 1545, 'sams-5b': 1315}

# tombo groups
tombo_groups = {'unm': 'RawGenomeCorrected_sams4f5',
                'm6A': 'RawGenomeCorrected_sams4f5',
                'vivo': 'RawGenomeCorrected_sams3abc4bdef5',
                'unm long': 'RawGenomeCorrected_sams3abc4bdef5'
                }

# csv file name
out_name = 'fast5_current_m6A_sams-4f5_100nt.csv'

In [17]:
# extract current data

# each sample
# container
current = pd.DataFrame()
for sample_name, fast5_path in fast5s.items():
    
    # load tombo-annotated reads
    tmb = tombo_helper.TomboReads([fast5_path], corrected_group=tombo_groups[sample_name])
    
    
    # each transcript
    df2 = pd.DataFrame()
    for transcript, position in plotSites.items():
        
        # get reads on a sams gene
        reads = tmb.get_cs_reads(chrm=transcript, strand='+')
        
        # shuffle reads
        random.shuffle(reads)


        # each read
        i = 0
        df1 = []
        for read in tqdm(reads, desc='reads', leave=False):
            
            # target region
            plotStart = position - read.start - region
            plotEnd = position - read.start + region
            
            # check read length
            if plotStart < 0 or read.end < (position + region):
                
                continue
                
            # get current data
            path = read.fn
            f5 = h5py.File(path, 'r')
            df1.append(f5['Analyses/' + tombo_groups[sample_name] + '/BaseCalled_template/Events'].value['norm_mean'][plotStart:plotEnd])
            
            # count (+ excess)
            if i == 3300:
                break
            i += 1
            
            
        df1 = pd.DataFrame(df1)

        # label transcript
        df1['sams'] = transcript
        
        df2 = df2.append(df1)
        
    
    # label sample
    df2['RNA'] = sample_name
    
    current = current.append(df2)
    
    
# output
current.to_csv(out_name, index=False)

[01:27:33] Parsing Tombo index file(s).


[01:28:04] Parsing Tombo index file(s).


[01:28:26] Parsing Tombo index file(s).


[01:28:27] Parsing Tombo index file(s).


In [11]:
current.shape

(15905, 104)

In [ ]:
# extract current data

# each sample
# container
current = pd.DataFrame()
for sample_name, fast5_path in fast5s[0].items():
    
    # load tombo-annotated reads
    tmb = tombo_helper.TomboReads([fast5_path], corrected_group=tombo_groups[sample_name])
    
    
    # each transcript
    df2 = pd.DataFrame()
    for transcript, position in plotSites[0].items():
        
        # get reads on a sams gene
        reads = tmb.get_cs_reads(chrm=transcript, strand='+')
        
        # shuffle reads
        random.shuffle(reads)


        # each read
        i = 0
        df1 = []
        for read in reads:
            
            # target region
            plotStart = position - read.start - region
            plotEnd = position - read.start + region
            
            # check read length
            if plotStart < 0 or read.end < (position + region):
                
                continue
                
            # get current data
            path = read.fn
            f5 = h5py.File(path, 'r')
            df1.append(f5['Analyses/' + tombo_groups[sample_name] + '/BaseCalled_template/Events'].value['norm_mean'][plotStart:plotEnd])
            
            # count (+ excess)
            if i == 3300:
                break
            i += 1
            
            
        df1 = pd.DataFrame(df1)

        # label transcript
        df1['sams'] = transcript
        
        df2 = df2.append(df1)
        
    
    # label sample
    df2['RNA'] = sample_name
    
    current = current.append(df2)
